In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

In [2]:
_=load_dotenv(find_dotenv())

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=0)

In [4]:
import os

# Función para leer los archivos a pasar a OpenAI
def process_files_in_folder(folder_path): # -> List[List[str]]:
    # Verificamos si la ruta proporcionada es una carpeta
    if not os.path.isdir(folder_path):
        print("La ruta proporcionada no es una carpeta válida.")
        return None

    data = []
    # Recorremos los archivos en la carpeta
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        # Verificamos si el elemento en la carpeta es un archivo
        if os.path.isfile(file_path):
            # Llamamos a la función textloader con la ruta del archivo
            loader = TextLoader(file_path)
            doc = loader.load()
            splits = text_splitter.split_documents(doc)
            data.append(splits)

    return data

# Ruta de la carpeta que contiene los archivos
folder_path = "files"

# Llamamos a la función para procesar los archivos en la carpeta
processed_data = process_files_in_folder(folder_path)

total_splits = []
# Ahora puedes hacer algo con los datos procesados, como imprimirlos
for document in processed_data:
    print(len(document))
    total_splits.extend(document)


1882
1219
52
714
46
2375
43
1947
4373
151


In [5]:
total_splits = []
for document in processed_data: # Crear un solo documento con todos los splits
    total_splits.extend(document)

In [6]:
len(total_splits)

12802

In [7]:
from langchain.vectorstores import Chroma
#Es una base de datos de embdings, te convierte tu texto en enveding y te lo guarda en una base de datos
#vectorstore = Chroma(
#    all_splits,
#    embedding_function = OpenAIEmbeddings(),
#    persist_directory = "test/chroma_db"
#)

vectorstore = Chroma.from_documents(total_splits, embedding=OpenAIEmbeddings())

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(temperature=0.0)
retriever = vectorstore.as_retriever()
memory = ConversationBufferMemory(
    llm=llm,
    memory_key= "chat_history",
    return_messages=True
)

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever = retriever,
    memory = memory
    
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
import gradio as gr

def chatbot(input_text):
    response = qa(input_text)
    return response["answer"]


with gr.Blocks() as demo:
        gr.Markdown(
    """
    # Empieza a chattear con el gymbot!
    """)

chatbot_interface = gr.Interface(fn=chatbot, 
                                 inputs="textbox", 
                                 outputs="textbox",
                                 description="Chatbot para responder preguntas sobre fitness",
                                 theme="soft",
                                 title="Gym Chatbot")

chatbot_interface.launch(share=True)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://95fddad9f194f779ff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
